In [1]:
## Global Static Array

In [ ]:
%%writefile 1.cu

#include <stdio.h>
#define N 10

__device__ int globalArray[N];

__global__ void incrementGlobalArray(){
    int i = threadIdx.x;

    if (i < N){
        globalArray[i]+=1;
    }
}

void initializeGlobalArray(int *hostArray){
    cudaMemcpyToSymbol(globalArray, hostArray, N * sizeof(int), 0, cudaMemcpyHostToDevice);
}

void retrieveGlobalArray(int *hostArray){
    cudaMemcpyFromSymbol(hostArray, globalArray, N * sizeof(int), 0, cudaMemcpyDeviceToHost);
}

int main(){
  int hostArray[N];

  for(int i=0; i < N; i++){
    hostArray[i] = i;
  }


  initializeGlobalArray(hostArray);

  incrementGlobalArray<<<1, N>>>();

  retrieveGlobalArray(hostArray);

  printf("Modified global Array: \n");

  for(int i=0; i<N; i++){
    printf("%d ", hostArray[i]);

  }
  printf("\n");

  return 0;
}


In [5]:
!nvcc -o 1 1.cu

In [6]:
!./1

Modified global Array: 
1 2 3 4 5 6 7 8 9 10 


In [2]:
## Assignment Q2

In [38]:
%%writefile 3.cu

#include <stdio.h>
#include <cuda_runtime.h>

#define SIZE 5

__device__ int d_a[SIZE];
__device__ int d_b[SIZE];
__device__ int d_c[SIZE];

__global__ void add_arrays() {
    int i = threadIdx.x;
    if (i < SIZE) {
        d_c[i] = d_a[i] + d_b[i];
    }
}

void initializeDeviceArrayA(int *a) {
    cudaMemcpyToSymbol(d_a, a, SIZE * sizeof(int), 0, cudaMemcpyHostToDevice);
}

void initializeDeviceArrayB(int *b) {
    cudaMemcpyToSymbol(d_b, b, SIZE * sizeof(int), 0, cudaMemcpyHostToDevice);
}

void retrieveDeviceArray(int *c) {
    cudaMemcpyFromSymbol(c, d_c, SIZE * sizeof(int), 0, cudaMemcpyDeviceToHost);
}

int main() {

    int a[SIZE] = {1, 2, 3, 4, 5};
    int b[SIZE] = {10, 9, 8, 7, 6};
    int c[SIZE];

    initializeDeviceArrayA(a);
    initializeDeviceArrayB(b);

    add_arrays<<<1, SIZE>>>();
    cudaDeviceSynchronize();

    retrieveDeviceArray(c);

    printf("Result:\n");
    for (int i = 0; i < SIZE; i++) {
        printf("%d ", c[i]);
    }
    printf("\n");

    return 0;
}


Overwriting 3.cu


In [39]:
!nvcc -o 3 3.cu

In [40]:
!./3

Result:
11 11 11 11 11 


In [23]:
%%writefile 5.cu

#include <stdio.h>
#define N 5

__device__ int d_a[N];
__device__ int d_b[N];
__device__ int d_c[N];

__global__ void AddArray(){
    int i = threadIdx.x;

    if (i < N){
        d_c[i]= d_a[i] + d_b[i];
    }
}


int main(){
  int h_a[N] = {1,2,3,4,5};
  int h_b[N] = {1,2,3,4,5};
  int h_c[N];

  cudaMemcpyToSymbol(d_a, h_a, N * sizeof(int), 0, cudaMemcpyHostToDevice);
  cudaMemcpyToSymbol(d_b, h_b, N * sizeof(int), 0, cudaMemcpyHostToDevice);


  AddArray<<<1, N>>>();
  cudaDeviceSynchronize();

  cudaMemcpyFromSymbol(h_c, d_c, N * sizeof(int), 0, cudaMemcpyDeviceToHost);


  for(int i=0; i<N; i++){
    printf("%d ", h_c[i]);

  }
  printf("\n");

  return 0;
}


Overwriting 5.cu


In [24]:
!nvcc -o 5 5.cu

In [25]:
!./5

2 4 6 8 10 


In [41]:
%%writefile 6.cu

#include <stdio.h>
#include <cuda_runtime.h>

int main(){
  cudaDeviceProp prop;
  cudaGetDeviceProperties(&prop, 0);

  printf("Compute capability: %d.%d\n", prop.major, prop.minor);
  printf("Max threads per block: %d\n", prop.maxThreadsPerBlock);
  printf("max blocks per dimension (x): %d\n", prop.maxGridSize[0]);
  printf("max threads per dim (x): %d\n", prop.maxThreadsDim[0]);

  return 0;
}

Writing 6.cu


In [42]:
!nvcc -o 6 6.cu

In [43]:
!./6

Compute capability: 7.5
Max threads per block: 1024
max blocks per dimension (x): 2147483647
max threads per dim (x): 1024


In [ ]:
%%writefile 5.cu

#include <stdio.h>
#include <cuda_runtime.h>

#define N 5

__device__ int d_c[N];

__global__ void AddArray(){
    int i = threadIdx.x;

    if (i < N){
        d_c[i]= d_a[i] + d_b[i];
    }
}


int main(){
  int h_a[N] = {1,2,3,4,5};
  int h_b[N] = {1,2,3,4,5};
  int h_c[N];

  int d_a[N], d_b[N];

  cudaMalloc((void **)&d_a, N * sizeof(int));
  cudaMalloc((void **)&d_b, N * sizeof(int));

  cudaMemcpy(d_a, h_a, N * sizeof(int), 0, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, h_b, N * sizeof(int), 0, cudaMemcpyHostToDevice);

  AddArray<<<1, N>>>();
  cudaDeviceSynchronize();

  cudaMemcpyFromSymbol(h_c, d_c, N * sizeof(int), 0, cudaMemcpyDeviceToHost);


  for(int i=0; i<N; i++){
    printf("%d ", h_c[i]);

  }
  printf("\n");

  cudaFree(d_a);
  cudaFree(d_b);

  return 0;
}
